In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
def open_files(date):
    focus_text = "2018-03-" + date + "_focus/2018-03-" + \
                 date + "_focus_group.json"
    krx_text = "krx_code.json"
    price_text = "2018-03-" + date + "_price_and_everything.json"
    
    with open(focus_text, 'r', encoding='UTF-8') as focus_group:
        focus = json.load(focus_group)
    with open(krx_text, 'r', encoding='UTF-8') as krx:
        krx_code = json.load(krx)
    with open(price_text, 'r', encoding='UTF-8') as pnc:
        price = json.load(pnc)
    
    return focus, krx_code, price

In [3]:
def get_time_disc(date):
    
    time = []
    prefix = "2018-03-" + date + "_focus/discussion_2018-03-" + date
    
    for hour in range(9,16):
        
        if date == "02" and hour == 9:
            continue
        
        elif date == "02" and hour == 10:
            continue
        
        elif date == "02" and hour == 15:
            continue
            
        for minute in range(0, 6):
            if date == "02" and hour == 11 and minute < 5:
                continue
            
            elif date == "02" and hour == 14 and minute > 4:
                continue
            
            time.append(prefix + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ \
                        ":" + str(minute) + "0.json")
            
            if hour == 15 and minute > 3:
                del time[len(time) - 1]
    
    return time

In [4]:
def df_list_dis(date):
    df_list = []
    time = get_time_disc(date)
    for x in time:
        with open(x, 'r', encoding='UTF-8') as f:
            discussion = json.load(f)        
            discuss_df = pd.DataFrame(discussion, columns = ["post_num", \
                                "unique_id", "click", "like", "dislike", 
                                "name", "time"])
            reset_col = ["name", "time", "post_num", "unique_id", "click", \
                         "like", "dislike"]
            discuss_df = discuss_df[reset_col]
            df_list.append(discuss_df)
    return df_list

In [5]:
'''
#half an hour high freq
def list_to_df(date, df_list, column_names, key_list):
    total = pd.DataFrame(columns=column_names)
    for ind, df in enumerate(df_list):
        if ind >= 5:
            df_total = df.merge(df_list[ind - 5], on = \
                                key_list).merge(df_list[ind - 4], \
                                on = key_list).merge(df_list[ind - 3], on = key_list)
            df_total.columns = column_names
            total = pd.concat([total, df_total], axis = 0)
    
    return total
'''

'\n#half an hour high freq\ndef list_to_df(date, df_list, column_names, key_list):\n    total = pd.DataFrame(columns=column_names)\n    for ind, df in enumerate(df_list):\n        if ind >= 5:\n            df_total = df.merge(df_list[ind - 5], on =                                 key_list).merge(df_list[ind - 4],                                 on = key_list).merge(df_list[ind - 3], on = key_list)\n            df_total.columns = column_names\n            total = pd.concat([total, df_total], axis = 0)\n    \n    return total\n'

In [6]:
#one hour high freq
def list_to_df(date, df_list, column_names, key_list):
    total = pd.DataFrame(columns=column_names)
    for ind, df in enumerate(df_list):
        if ind >= 8:
            df_total = df.merge(df_list[ind - 8], on = \
                                key_list).merge(df_list[ind - 7], \
                                on = key_list).merge(df_list[ind - 6], on = key_list)
            df_total.columns = column_names
            total = pd.concat([total, df_total], axis = 0)
    
    return total

In [7]:
COLUMN_DISC =["name", "time", "post_num", "unique_id", "click", \
              "like", "dislike", "time_1", "post_num_1", "unique_id_1", "click_1", \
              "like_1", "dislike_1", "time_2", "post_num_2", "unique_id_2", "click_2", \
              "like_2", "dislike_2", "time_3", "post_num_3", "unique_id_3", "click_3", \
              "like_3", "dislike_3"]

def get_discuss_df(date):
    discussion_list = df_list_dis(date)
    discuss_df = list_to_df(date, discussion_list, COLUMN_DISC, 
                            ['name']).reset_index().drop(["index"], axis = 1)
    return discuss_df

In [8]:
def df_list_price(date):
    price_df = pd.DataFrame(price, columns = ["index", "code", "name", "time", "price", \
                                         "price_dif", "sell", "buy", "volume", "variation"])
    price_df = price_df[["code", "name", "time", "price", \
                    "price_dif", "sell", "buy", "volume", "variation"]][1:]
    text = "2018-03-" + date + " 09:00"
    price_df = price_df[price_df["time"] != text]
    time_list = price_df['time'].unique().tolist()
    price_df_list = []
    for time in time_list:
        df = price_df[price_df["time"] == time]
        df = df[["code", "name", "time", "price", \
                "price_dif", "sell", "buy", "volume", "variation"]]
        price_df_list.append(df)
        
    return price_df_list

In [9]:
COLUMN_PRICE=["code", "name", "time", "price", "price_dif", "sell", 
              "buy", "volume", "variation", "time_1", "price_1", 
              "price_dif_1", "sell_1", "buy_1", "volume_1", "variation_1",
              "time_2", "price_2", "price_dif_2", "sell_2", "buy_2", 
              "volume_2", "variation_2", "time_3", "price_3", 
              "price_dif_3", "sell_3", "buy_3", "volume_3", "variation_3"]

def get_price_df(date):
    price_df_list = df_list_price(date)
    price_df = list_to_df(date, price_df_list, COLUMN_PRICE, \
                          ['code', 'name']).reset_index().drop(["index"], axis = 1)
    
    return price_df

In [10]:
COLUMN_TOTAL = ['name', 'code', 'time', 'price', 'time_1', "price_1", \
                "price_dif_1", "sell_1", "buy_1", "volume_1", "variation_1", \
                'post_num_1', 'unique_id_1', 'click_1', 'like_1', 'dislike_1', \
                'time_2', 'price_2', "price_dif_2", "sell_2", "buy_2", \
                "volume_2", "variation_2", 'post_num_2', 'unique_id_2', 'click_2', \
                'like_2', 'dislike_2', 'time_3', 'price_3', "price_dif_3", \
                "sell_3", "buy_3", "volume_3", "variation_3", 'post_num_3', \
                'unique_id_3', 'click_3', 'like_3', 'dislike_3']

def get_total_df(date):
    price_df = get_price_df(date)
    discuss_df = get_discuss_df(date)
    total_df = pd.merge(price_df, discuss_df, on = ['name', \
                                                    'time', 'time_1', 'time_2', 'time_3'])
    total_df = total_df[COLUMN_TOTAL]
    
    return total_df

In [11]:
import numpy as np
import re

with open("company_size.json", 'r', encoding='UTF-8') as f:
    company_size = json.load(f)

company_df = pd.DataFrame({"company_name": np.zeros(0), \
                           "code": np.zeros(0), "market": np.zeros(0), 
                          "company_size": np.zeros(0)})

company_df = pd.DataFrame(company_size, columns = ["name", "code", "market", "size"])

KOSPI = []
KOSDAQ = []
TRASH = []
MKT_CAP = {}

for idx, row in company_df.iterrows():
    if re.sub('[0-9 ,위]', '', row["market"]) == "코스피":
        KOSPI.append(row["code"])
    elif re.sub('[0-9 ,위]', '', row["market"]) == "코스닥":
        KOSDAQ.append(row["code"])
    else:
        TRASH.append(row["code"])
    
    MKT_CAP[row["code"]] = int(re.sub('[조억원,]', '', row["size"]))

In [12]:
def add_company(date):
    
    total = get_total_df(date)
    total["mkt_cap"] = np.nan
    total["kospi"] = np.nan
    total["kosdaq"] = np.nan
    total["trash"] = np.nan
    
    for index, row in total.iterrows():
        mkt_cap = MKT_CAP[row["code"]]
        if row["code"] in KOSPI:
            kospi = 1
        else:
            kospi = 0

        if row["code"] in KOSDAQ:
            kosdaq = 1
        else:
            kosdaq = 0

        if row["code"] in TRASH:
            trash = 1
        else:
            trash = 0
        total.set_value(index,'mkt_cap', mkt_cap)
        total.set_value(index,'kospi', kospi)
        total.set_value(index,'kosdaq', kosdaq)
        total.set_value(index,'trash', trash)  
    
    return total

In [13]:
def transform_df(date):
    
    total = add_company(date)
    total.dropna(inplace = True)
    var_to_transform = ['price', 'price_1', 'price_dif_1', 'sell_1', 
                        'buy_1', 'volume_1', 'variation_1', 'price_2', 
                        'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 
                        'variation_2', 'price_3', 'price_dif_3', 'sell_3', 'buy_3', 
                        'volume_3', 'variation_3']
    for var in var_to_transform:
        total = total[total[var] != '\xa0']
        for index, row in total.iterrows():
            if isinstance(row[var], str):
                data = row[var].split(",")
                value = ''.join(data)
                value = int(value)
                total.set_value(index, var, value)
    return total

In [14]:
focus, krx_code, price = open_files('02')
total_02 = transform_df('02')

In [15]:
'''del_time = ["2018-02-27 11:30", "2018-02-27 11:40", "2018-02-27 11:50", "2018-02-27 12:00", "2018-02-27 12:10", \
            "2018-02-27 12:20", "2018-02-27 12:30", "2018-02-27 12:40", "2018-02-27 12:50", "2018-02-27 13:00", \
            "2018-02-27 13:10", "2018-02-27 13:20", "2018-02-27 13:30", "2018-02-27 13:40"]
total_27 = total_27[~total_27['time'].isin(del_time)]'''

'del_time = ["2018-02-27 11:30", "2018-02-27 11:40", "2018-02-27 11:50", "2018-02-27 12:00", "2018-02-27 12:10",             "2018-02-27 12:20", "2018-02-27 12:30", "2018-02-27 12:40", "2018-02-27 12:50", "2018-02-27 13:00",             "2018-02-27 13:10", "2018-02-27 13:20", "2018-02-27 13:30", "2018-02-27 13:40"]\ntotal_27 = total_27[~total_27[\'time\'].isin(del_time)]'

In [16]:
total_df = pd.concat([total_02])

In [17]:
total_df = total_df.reset_index().drop(["index"], axis = 1)

In [18]:
total_df["yesterday_closing_price"] = total_df["price_1"] - total_df["price_dif_1"]

In [19]:
total_df["is_maximum"] = (((total_df["price_1"] / total_df["yesterday_closing_price"]) - 1) * 100 > 29.5) | \
(((total_df["price_2"] / total_df["yesterday_closing_price"]) - 1) * 100 > 29.5) | \
(((total_df["price_3"] / total_df["yesterday_closing_price"]) - 1) * 100 > 29.5)
total_df["is_maximum"] = total_df["is_maximum"].astype(int)

In [20]:
total_df["is_minimum"] = (((total_df["price_1"] / total_df["yesterday_closing_price"]) - 1) * 100 < -29.5) | \
(((total_df["price_2"] / total_df["yesterday_closing_price"]) - 1) * 100 < -29.5) | \
(((total_df["price_3"] / total_df["yesterday_closing_price"]) - 1) * 100 < -29.5)
total_df["is_minimum"] = total_df["is_maximum"].astype(int)

In [21]:
total_df["price_volatility"] = (((total_df[["price_1", "price_2", "price_3"]].max(axis=1)) / (total_df[["price_1", "price_2", "price_3"]].min(axis=1))) - 1) * 100

In [22]:
total_df["price_trend"] = (((total_df["price_1"] - total_df["price_2"]) < 0).astype(int) + ((total_df["price_2"] - total_df["price_3"]) < 0).astype(int)) - (((total_df["price_1"] - total_df["price_2"]) > 0).astype(int) + ((total_df["price_2"] - total_df["price_3"]) > 0).astype(int))

In [23]:
total_df["average_price_volatility"] = total_df["price_trend"] * total_df["price_volatility"] / 2

In [24]:
total_df["sell_minus_buy_1"] = total_df["sell_1"] - total_df["buy_1"]

In [25]:
total_df["sell_minus_buy_2"] = total_df["sell_2"] - total_df["buy_2"]

In [26]:
total_df["sell_minus_buy_3"] = total_df["sell_3"] - total_df["buy_3"]

In [27]:
total_df["is_price_gap_stable"] = ((total_df["sell_minus_buy_1"] == total_df["sell_minus_buy_2"]) & (total_df["sell_minus_buy_2"] == total_df["sell_minus_buy_3"])).astype(int)

In [28]:
total_df["price_gap_volatility"] = (((total_df[["sell_minus_buy_1", "sell_minus_buy_2", "sell_minus_buy_3"]].max(axis=1)) / (total_df[["sell_minus_buy_1", "sell_minus_buy_2", "sell_minus_buy_3"]].min(axis=1)))) - 1

In [29]:
total_df["is_like_higher"] = (total_df["like_3"] > total_df["dislike_3"]).astype(int)

In [30]:
total_df["volume_trend"] = ((((total_df["volume_3"]) - (total_df["volume_2"] * 1+1e-3)) \
 / ((total_df["volume_2"]) - (total_df["volume_1"] * 1-1e-4))) - 1) * 100

In [31]:
total_df["post_num_trend"] = ((((total_df["post_num_3"]) - (total_df["post_num_2"] * 1+1e-3)) \
 / ((total_df["post_num_2"]) - (total_df["post_num_1"] * 1-1e-4))) - 1) * 100

In [32]:
total_df["unique_id_trend"] = ((((total_df["unique_id_3"]) - (total_df["unique_id_2"] * 1+1e-3)) \
 / ((total_df["unique_id_2"]) - (total_df["unique_id_1"] * 1-1e-4))) - 1) * 100

In [33]:
total_df["click_trend"] = ((((total_df["click_3"]) - (total_df["click_2"] * 1+1e-3)) \
 / ((total_df["click_2"]) - (total_df["click_1"] * 1-1e-4))) - 1) * 100

In [34]:
total_df["price_increase"] = ((total_df["price"] / total_df["price_3"]) - 1) * 100

In [35]:
total_df["did_price_increase"] = (total_df["price_increase"] > 0).astype(int)

In [36]:
total_df["did_price_033"] = (total_df["price_increase"] > 0.33).astype(int)

In [37]:
total_df["did_price_100"] = (total_df["price_increase"] > 1.0).astype(int)

In [38]:
total_df["did_price_150"] = (total_df["price_increase"] > 1.5).astype(int)

In [39]:
with open('KOSPI_Feb14_Mar02.json', 'r', encoding='UTF-8') as f:
    kospi = json.load(f)

In [40]:
with open('KOSDAQ_Feb14_Mar02.json', 'r', encoding='UTF-8') as f:
    kosdaq = json.load(f)

In [41]:
with open('kospi_now_Feb14_Mar02.json', 'r', encoding='UTF-8') as f:
    kospi_now = json.load(f)

In [42]:
with open('kosdaq_now_Feb14_Mar02.json', 'r', encoding='UTF-8') as f:
    kosdaq_now = json.load(f)

In [43]:
total_df["kospi_ind"] = np.nan
total_df["kosdaq_ind"] = np.nan
total_df["time_slot"] = np.nan
total_df["ko_inter"] = np.nan
total_df["early_mor"] = np.nan
total_df["morning"] = np.nan
total_df["lunch"] = np.nan
total_df["afternoon"] = np.nan
total_df["late"] = np.nan
total_df["mkt_change"] = np.nan
total_df["alpha"] = np.nan
total_df['per_now'] = np.nan

total_df["kospi_1"] = np.nan
total_df["kospi_2"] = np.nan
total_df["kospi_3"] = np.nan
total_df["kospi_answer"] = np.nan

total_df["kosdaq_1"] = np.nan
total_df["kosdaq_2"] = np.nan
total_df["kosdaq_3"] = np.nan
total_df["kosdaq_answer"] = np.nan


num = 1
time_list = []
for index, row in total_df.iterrows():
    pi_ind = kospi[row['time']]
    daq_ind = kosdaq[row['time']]
    total_df.set_value(index,'kospi_ind', pi_ind)
    total_df.set_value(index,'kosdaq_ind', daq_ind)
    time = row['time'].split()[1]
    if time not in time_list:
        time_list.append(time)
    ind = time_list.index(time)
    num = index + 1
    
    if num in [1, 2, 3]:
        early_mor = 1
    else:
        early_mor = 0
    
    if (num >= 1) and (num <= 15):
        morning = 1
    else:
        morning = 0
        
    if (num >= 16) and (num <= 24):
        lunch = 1
    else:
        lunch = 0
    
    if (num >= 25) and (num <= 36):
        afternoon = 1
    else:
        afternoon = 0
    
    if (num >= 25) and (num <= 36):
        afternoon = 1
    else:
        afternoon = 0
    
    if (num >= 31) and (num <= 36):
        late = 1
    else:
        late = 0
    

    mkt_change = row['kospi'] * pi_ind + row['kosdaq'] * daq_ind
    total_df.set_value(index,'mkt_change', mkt_change)
    total_df.set_value(index,'early_mor', early_mor)
    total_df.set_value(index,'morning', morning)
    total_df.set_value(index,'lunch', lunch)
    total_df.set_value(index,'afternoon', afternoon)
    total_df.set_value(index,'late', late)
    total_df.set_value(index,'time_slot', num)
    total_df.set_value(index,'ko_inter', pi_ind * daq_ind)
    per_now = 100 * row['price_dif_3']/row['yesterday_closing_price']
    total_df.set_value(index, 'per_now', per_now)
    alpha = per_now - mkt_change
    total_df.set_value(index, 'alpha', alpha)
    
    total_df.set_value(index, 'kospi_1', kospi_now[row["time_1"]])
    total_df.set_value(index, 'kospi_2', kospi_now[row["time_2"]])
    total_df.set_value(index, 'kospi_3', kospi_now[row["time_3"]])
    total_df.set_value(index, 'kospi_answer', kospi_now[row["time"]])
    
    total_df.set_value(index, 'kosdaq_1', kosdaq_now[row["time_1"]])
    total_df.set_value(index, 'kosdaq_2', kosdaq_now[row["time_2"]])
    total_df.set_value(index, 'kosdaq_3', kosdaq_now[row["time_3"]])
    total_df.set_value(index, 'kosdaq_answer', kosdaq_now[row["time"]])

In [44]:
total_df["kospi_trend"] = ((((total_df["kospi_3"]) - (total_df["kospi_2"] * 1+1e-3)) \
 / ((total_df["kospi_2"]) - (total_df["kospi_1"] * 1-1e-4))) - 1) * 100

total_df["kosdaq_trend"] = ((((total_df["kosdaq_3"]) - (total_df["kosdaq_2"] * 1+1e-3)) \
 / ((total_df["kosdaq_2"]) - (total_df["kosdaq_1"] * 1-1e-4))) - 1) * 100

total_df["kospi_increase"] = 100 * (total_df["kospi_answer"] - total_df["kospi_3"]) / total_df["kospi_3"]

total_df["kosdaq_increase"] = 100 * (total_df["kosdaq_answer"] - total_df["kosdaq_3"]) / total_df["kosdaq_3"]

total_df["market_increase"] = (total_df["kospi"] * total_df["kospi_increase"]) \
                            + (total_df["kosdaq"] * total_df["kosdaq_increase"])

In [45]:
with open('2018-03-02_opening_increase.json', 'r', encoding='UTF-8') as f:
    increase_02 = json.load(f)

mar_opening = {"02": increase_02}

In [46]:
total_df["did_opening_price_increase"] = np.nan

In [47]:
for index, row in total_df.iterrows():
    if row["name"] in mar_opening[row["time"][8:10]]:
        total_df.set_value(index,'did_opening_price_increase', 1)
    else:
        total_df.set_value(index,'did_opening_price_increase', 0)

In [48]:
TO_SQ = ['price_1', 'price_dif_1', 'sell_1', 'buy_1', 'volume_1', 'variation_1', 'post_num_1', 'unique_id_1', 'click_1', 'like_1', 'dislike_1', 'price_2', 'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 'variation_2', 'post_num_2', 'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'price_3', 'price_dif_3', 'sell_3', 'buy_3', 'volume_3', 'variation_3', 'post_num_3', 'unique_id_3', 'click_3', 'like_3', 'dislike_3', 'mkt_cap', 'yesterday_closing_price', 'price_volatility', 'price_trend', 'average_price_volatility', 'sell_minus_buy_1', 'sell_minus_buy_2', 'sell_minus_buy_3', 'price_gap_volatility', 'volume_trend', 'post_num_trend', 'unique_id_trend', 'click_trend', 'kospi_ind', 'kosdaq_ind', 'time_slot', 'ko_inter', 'mkt_change', 'alpha', 'per_now', 'kospi_1', 'kospi_2', 'kospi_3', 'kosdaq_1', 'kosdaq_2', 'kosdaq_3', 'kospi_trend', 'kosdaq_trend']

In [49]:
for var in TO_SQ:
    col_name = var + '_sq'
    total_df[col_name] = np.nan
    for index, row in total_df.iterrows():
        sqr = row[var] ** 2
        total_df.set_value(index, col_name, sqr)

In [50]:
print(len(total_df))
total_df = total_df.dropna(axis=0, how='any')
print(len(total_df))

180
180


In [51]:
X = list(total_df.columns)
print(X)

['name', 'code', 'time', 'price', 'time_1', 'price_1', 'price_dif_1', 'sell_1', 'buy_1', 'volume_1', 'variation_1', 'post_num_1', 'unique_id_1', 'click_1', 'like_1', 'dislike_1', 'time_2', 'price_2', 'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 'variation_2', 'post_num_2', 'unique_id_2', 'click_2', 'like_2', 'dislike_2', 'time_3', 'price_3', 'price_dif_3', 'sell_3', 'buy_3', 'volume_3', 'variation_3', 'post_num_3', 'unique_id_3', 'click_3', 'like_3', 'dislike_3', 'mkt_cap', 'kospi', 'kosdaq', 'trash', 'yesterday_closing_price', 'is_maximum', 'is_minimum', 'price_volatility', 'price_trend', 'average_price_volatility', 'sell_minus_buy_1', 'sell_minus_buy_2', 'sell_minus_buy_3', 'is_price_gap_stable', 'price_gap_volatility', 'is_like_higher', 'volume_trend', 'post_num_trend', 'unique_id_trend', 'click_trend', 'price_increase', 'did_price_increase', 'did_price_033', 'did_price_100', 'did_price_150', 'kospi_ind', 'kosdaq_ind', 'time_slot', 'ko_inter', 'early_mor', 'morning', 'lunch', 'afte

In [53]:
#total_df.to_json('df_1hour_Mar.json', orient='values')